# Evaluation


In [1]:
with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [2]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


alberto-lorente


In [3]:
import torch

device = "cpu"
if torch.cuda.is_available():
    print("Cuda available")
    device = torch.device('cuda')

Cuda available


### Preparing the Data for Evaluation

In [4]:
import os 
eval_path = r"Evaluation data\Solved PDFs"
solved_pdfs_path = os.listdir(eval_path)
solved_pdfs_path_pdf_files = [os.path.join(eval_path, pdf) for pdf in solved_pdfs_path if pdf.endswith("pdf")]
solved_pdfs_path_txt_files = [os.path.join(eval_path, txt) for txt in solved_pdfs_path if txt.endswith("txt")]

In [5]:
list_tuple_data = []
i = 0
while i < len(solved_pdfs_path_pdf_files):
    tup = (solved_pdfs_path_pdf_files[i], solved_pdfs_path_txt_files[i])
    list_tuple_data.append(tup)
    i += 1

In [6]:
list_tuple_data

[('Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.pdf',
  'Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.txt'),
 ('Evaluation data\\Solved PDFs\\7a3ef5c116cca7f5328c44ef2c3f227f8c90ef60_Zones_Accélérati.pdf',
  'Evaluation data\\Solved PDFs\\7a3ef5c116cca7f5328c44ef2c3f227f8c90ef60_Zones_Accélérati.txt'),
 ('Evaluation data\\Solved PDFs\\a7f4c9d0d7c8f3c9cd67b7c80beec26d9c234382_delibs-22-mai-24.pdf',
  'Evaluation data\\Solved PDFs\\a7f4c9d0d7c8f3c9cd67b7c80beec26d9c234382_delibs-22-mai-24.txt'),
 ('Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.pdf',
  'Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.txt'),
 ('Evaluation data\\Solved PDFs\\fa224c7892c9c4971dda6423c75c97a04f6e3666_del2024_22-defin.pdf',
  'Evaluation data\\Solved PDFs\\fa224c7892c9c4971dda6423c75c97a04f6e3666_del2024_22-defin.txt')]

### Loading Prompts

In [7]:
import json

with open("prompts_preprocessing.json", "r") as f:
    prompts_for_processing = json.load(f)

table_process_prompt = prompts_for_processing["augment_table_prompt"]
summary_prompt = prompts_for_processing["summary_prompt"]

In [8]:
with open("french_prompts.json", "r") as f:
    prompts_query = json.load(f)

## Loading markdown

In [9]:
pdf_path = list_tuple_data[-2][0]
markdown_path = list_tuple_data[-2][1]

In [10]:
pdf_path

'Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.pdf'

In [11]:
with open(markdown_path, "r") as f:
    markdown = f.read()

In [12]:
markdown

'# Réunion du Conseil Municipal\n\n# du 12 juillet 2023\n\nLe douze juillet deux mil vingt-trois, à 20 heures, les membres du Conseil Municipal se sont réunis à la mairie de la commune de Coulon sous la présidence de Madame Anne-Sophie GUICHET, Maire.\n\nÉtaient présents : Mmes et Mrs Fabrice BERJONNEAU, Juliette DELAVALLE, Angélique DUMOULIN, Dominique GIRET, Julien GUIBERT, Anne-Sophie GUICHET, Isabelle HÉHUNSTRE, Marie LE CHAPELAIN, Virginie LEONARD, Line MARCHÉ, Béatrice MORIN, Mélanie MOUSSION, François SABOURIN.\n\nÉtaient absents et excusés Patrick CARTIER (pouvoir à Isabelle HÉHUNSTRE), Vaianu FENUAITI (pouvoir à Anne-Sophie GUICHET), Benoît LALÈRE (pouvoir à Julien GUIBERT), Pascal MORIN, Romain MORIN, Stéphane RICHARD (pouvoir à Dominique GIRET).\n\nDate de convocation : 06 juillet 2023\n\nSecrétaire de séance : Fabrice BERJONNEAU\n\nAucune observation n’étant formulée, le compte rendu de la précédente réunion est approuvé à l’unanimité.\n\nAvant d’aborder l’ordre du jour, Mm

## Processing

In [13]:
from council_rag.preprocessing import preprocess_markdown_text
from council_rag.data_transformations import process_tables, summarize_clusters
from council_rag.preprocessing.preprocessing import unload_cuda

import time

unload_cuda()
start = time.time()

paragraphs_list, clusters_dict, model = preprocess_markdown_text(markdown,
                                                        model_id ="Jaume/gemma-2b-embeddings", 
                                                        spacy_model="fr_core_news_sm", 
                                                        n_sents_per_para=8,
                                                        device=device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.34it/s]


In [14]:
unload_cuda()
processed_tables = process_tables(pdf_path,
                                eval_path, # required to know where to save the pdf images
                                table_process_prompt, 
                                groq_token)

In [15]:
unload_cuda()
clusters_dict = summarize_clusters(clusters_dict, 
                                    summary_prompt, 
                                    groq_token, 
                                    model="gemma2-9b-it", 
                                    token_limit=14000, 
                                    sleep_time=60)

end = time.time()
print((end - start)/60)

3.6205443143844604


In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from council_rag.rag import prepare_data_for_rag

unload_cuda()
clusters_list, paragraphs_list, all_docs = prepare_data_for_rag(clusters_dict, 
                                                                paragraphs_list, 
                                                                processed_tables, 
                                                                splitter=RecursiveCharacterTextSplitter,
                                                                chunk_size=450,
                                                                chunk_overlap=35,
                                                                length_function=len,
                                                                is_separator_regex=False)

Cuda available


In [ ]:
from council_rag.rag import shorten_summary_docs

unload_cuda()
all_docs = shorten_summary_docs(all_docs, groq_token)

In [ ]:
from council_rag.preprocessing.preprocessing import compute_norm_embeddings

In [20]:
all_embeddings = []
for doc in all_docs:
    unload_cuda()
    embed = model.encode(doc.page_content)
    all_embeddings.append(embed)